## Recommendation using FFM

Uses a Field-Aware Factorization Machine to recommend hotels on the Las Vegas Strip for the selected traveler type and season, based on 2015 TripAdvisor ratings.

### NuGet Packages

In [1]:
#r "nuget:Microsoft.ML, 1.4.0"
#r "nuget:Microsoft.ML.Recommender, 0.16.0"
#r "nuget:XPlot.Plotly, 3.0.1"

Installing package Microsoft.ML, version 1.4.0.........................done!

Successfully added reference to package Microsoft.ML, version 1.4.0

Installing package Microsoft.ML.Recommender, version 0.16.0.......done!

Successfully added reference to package Microsoft.ML.Recommender, version 0.16.0

Installing package XPlot.Plotly, version 3.0.1.......done!

Successfully added reference to package XPlot.Plotly, version 3.0.1

### Namespaces

In [3]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using XPlot.Plotly;
using System.IO;
using System.Linq;

### Input class definition

In [4]:
public class FfmRecommendationData
{
    public bool Label;

    public string TravelerType;

    public string Season;

    public string Hotel;
}

### Output class definition

In [5]:
public class FfmRecommendationPrediction
{
    public bool PredictedLabel;

    public float Probability;

    public string TravelerType;

    public string Season;

    public string Hotel;
}

### Read the raw data

In [8]:
var mlContext = new MLContext(seed: null);

var ratingTreshold = 3;

// Populating an IDataView from an IEnumerable.
var data = File.ReadAllLines("./LasVegasTripAdvisorReviews.csv")
    .Skip(1)
    .Select(x => x.Split(';'))
    .Select(x => new FfmRecommendationData
    {
        Label = double.Parse(x[4]) > ratingTreshold,
        Season = x[5],
        TravelerType = x[6],
        Hotel = x[13]
    });

var allData = mlContext.Data.LoadFromEnumerable(data);

// return _mlContext.Data.CreateEnumerable<FfmRecommendationData>(_allData, reuseRowObject: false);


### Prepare the data

In [9]:
var pipeline = mlContext.Transforms.Categorical.OneHotEncoding("TravelerTypeOneHot", "TravelerType")
    .Append(mlContext.Transforms.Categorical.OneHotEncoding("SeasonOneHot", "Season"))
    .Append(mlContext.Transforms.Categorical.OneHotEncoding("HotelOneHot", "Hotel"))
    .Append(mlContext.Transforms.Concatenate("Features", "TravelerTypeOneHot", "SeasonOneHot", "HotelOneHot"))
    .Append(mlContext.BinaryClassification.Trainers.FieldAwareFactorizationMachine(new string[] { "Features" }));

### Train the model

In [10]:
var trainingData = mlContext.Data.ShuffleRows(allData);
trainingData = mlContext.Data.TakeRows(trainingData, 450);

var model = pipeline.Fit(trainingData);

### Evaluate the model

In [11]:
var testData = mlContext.Data.ShuffleRows(allData);
testData = mlContext.Data.TakeRows(testData, 100);

var scoredData = model.Transform(testData);
var metrics = mlContext.BinaryClassification.Evaluate(
    data: scoredData,
    labelColumnName: "Label",
    scoreColumnName: "Probability",
    predictedLabelColumnName: "PredictedLabel");
    
display(metrics);

LogLoss,LogLossReduction,Entropy,AreaUnderRocCurve,Accuracy,PositivePrecision,PositiveRecall,NegativePrecision,NegativeRecall,F1Score,AreaUnderPrecisionRecallCurve,ConfusionMatrix
0.6078563394217641,0.13345535180808038,0.7014714598838974,0.7605588044184536,0.81,0.81,1,0,0,0.8950276243093923,0.9338675518491129,"{ Microsoft.ML.Data.ConfusionMatrix: PerClassPrecision: [ 0.81, 0 ], PerClassRecall: [ 1, 0 ], Counts: [ [ 81, 0 ], [ 19, 0 ] ], NumberOfClasses: 2 }"


### Values

In [13]:
var hotels = data.Select(r => r.Hotel).Distinct().ToList();
hotels.Sort();
display (hotels);

var travelerTypes = data.Select(r => r.TravelerType).Distinct().ToList();
travelerTypes.Sort();
display (travelerTypes);

var seasons = data.Select(r => r.Season).Distinct().ToList();
seasons.Sort();
display (seasons);

index,value
0,Bellagio Las Vegas
1,Caesars Palace
2,Circus Circus Hotel & Casino Las Vegas
3,Encore at wynn Las Vegas
4,Excalibur Hotel & Casino
5,Hilton Grand Vacations at the Flamingo
6,Hilton Grand Vacations on the Boulevard
7,Marriott's Grand Chateau
8,Monte Carlo Resort&Casino
9,Paris Las Vegas


index,value
0,Business
1,Couples
2,Families
3,Friends
4,Solo


index,value
0,Dec-Feb
1,Jun-Aug
2,Mar-May
3,Sep-Nov


### Single predicition

In [19]:
var predictionEngine = mlContext.Model.CreatePredictionEngine<FfmRecommendationData, FfmRecommendationPrediction>(model);
var prediction = predictionEngine.Predict(new FfmRecommendationData
    { 
        TravelerType = "Couples",
        Hotel = "Paris Las Vegas",
        Season = "Mar-May"
    });

display(prediction);

PredictedLabel,Probability,TravelerType,Season,Hotel
True,0.73002464,Couples,Mar-May,Paris Las Vegas


### Bulk prediction

In [28]:
var input = hotels.Select(h => new FfmRecommendationData
               {
                   TravelerType = "Friends",
                   Season = "Jun-Aug",
                   Hotel = h
               }).ToList();
var predictions = model.Transform(mlContext.Data.LoadFromEnumerable(input));
var result = mlContext.Data.CreateEnumerable<FfmRecommendationPrediction>(predictions, reuseRowObject: false);
result = result.Where(r => r.PredictedLabel == true).OrderByDescending(r => r.Probability);

display(result);

index,PredictedLabel,Probability,TravelerType,Season,Hotel
0,True,0.9604928,Friends,Jun-Aug,Marriott's Grand Chateau
1,True,0.9533174,Friends,Jun-Aug,Encore at wynn Las Vegas
2,True,0.95096093,Friends,Jun-Aug,The Venetian Las Vegas Hotel
3,True,0.94756126,Friends,Jun-Aug,Wynn Las Vegas
4,True,0.93594223,Friends,Jun-Aug,Wyndham Grand Desert
5,True,0.9153716,Friends,Jun-Aug,Trump International Hotel Las Vegas
6,True,0.9151229,Friends,Jun-Aug,The Cosmopolitan Las Vegas
7,True,0.908976,Friends,Jun-Aug,Hilton Grand Vacations on the Boulevard
8,True,0.9076306,Friends,Jun-Aug,Bellagio Las Vegas
9,True,0.8987,Friends,Jun-Aug,Treasure Island- TI Hotel & Casino


### Visualization

In [24]:
var graph = new Graph.Bar()
{
    y = result.Select(r => r.Probability),
    x = result.Select(r => r.Hotel),
    marker = new Graph.Marker { color = "darkred" }
};

var chart = Chart.Plot(graph);

var layout = new Layout.Layout(){ title="Recommended Hotels for Friends in Summer" };
chart.WithLayout(layout);

display(chart);